In [1]:
!pip install langextract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.2/106.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 4.8 MB/s eta 0:00:00


In [3]:
!pip install docling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.6/86.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 30.7 MB/s eta 0:00:00
   ━━━━

In [22]:
from google.colab import userdata
api_key=userdata.get('GOOGLE_API_KEY')

In [28]:
import langextract as le
from docling.document_converter import DocumentConverter
from IPython.display import display, HTML


In [56]:
converter=DocumentConverter()
doc = converter.convert("File.pdf")
text=doc.document.export_to_text()

In [15]:
prompt="""
You are an information extraction system.
Extract all named entities from the input text and return them in strict JSON format.
The entity classes to use are:
- ORG → organizations, companies, institutions, agencies
- PERSON → people
- GPE → geopolitical entities (countries, cities, states, regions)
- DATE → calendar dates, years, time periods
- AMOUNT → monetary values (e.g., $5, 100 euros, Rs. 10 lakh)
- QTY → quantities or numerical measures (e.g., 5 kg, 3 km, 200 people)

For each entity, provide:
- text → the exact substring from the source
- start → starting character index in the input text
- end → ending character index in the input text
- class → one of [ORG, PERSON, GPE, DATE, AMOUNT, QTY]

return valid JSON of the extracted entities in order of appearance.
"""

In [16]:
examples = [
    le.data.ExampleData(
        text="Google was founded by Larry Page and Sergey Brin in California in 1998 with $100,000 funding.",
        extractions=[
            le.data.Extraction(
                extraction_class="ORG",
                extraction_text="Google",
                attributes={"start": 0, "end": 6}
            ),
            le.data.Extraction(
                extraction_class="PERSON",
                extraction_text="Larry Page",
                attributes={"start": 21, "end": 31}
            ),
            le.data.Extraction(
                extraction_class="PERSON",
                extraction_text="Sergey Brin",
                attributes={"start": 36, "end": 47}
            ),
            le.data.Extraction(
                extraction_class="GPE",
                extraction_text="California",
                attributes={"start": 51, "end": 61}
            ),
            le.data.Extraction(
                extraction_class="DATE",
                extraction_text="1998",
                attributes={"start": 65, "end": 69}
            ),
            le.data.Extraction(
                extraction_class="AMOUNT",
                extraction_text="$100,000",
                attributes={"start": 76, "end": 84}
            )
        ]
    ),
    le.data.ExampleData(
        text="Amazon hired 200 employees in India in June 2023.",
        extractions=[
            le.data.Extraction(
                extraction_class="ORG",
                extraction_text="Amazon",
                attributes={"start": 0, "end": 6}
            ),
            le.data.Extraction(
                extraction_class="QTY",
                extraction_text="200 employees",
                attributes={"start": 13, "end": 26}
            ),
            le.data.Extraction(
                extraction_class="GPE",
                extraction_text="India",
                attributes={"start": 30, "end": 35}
            ),
            le.data.Extraction(
                extraction_class="DATE",
                extraction_text="June 2023",
                attributes={"start": 39, "end": 48}
            )
        ]
    ),
    le.data.ExampleData(
        text="The United Nations signed the treaty in Paris on 10 December 1948.",
        extractions=[
            le.data.Extraction(
                extraction_class="ORG",
                extraction_text="United Nations",
                attributes={"start": 4, "end": 18}
            ),
            le.data.Extraction(
                extraction_class="GPE",
                extraction_text="Paris",
                attributes={"start": 41, "end": 46}
            ),
            le.data.Extraction(
                extraction_class="DATE",
                extraction_text="10 December 1948",
                attributes={"start": 50, "end": 66}
            )
        ]
    ),
    le.data.ExampleData(
        text="The company Infosys spent Rs. 5 lakh for 200 chairs in Bangalore.",
        extractions=[
            le.data.Extraction(
                extraction_class="ORG",
                extraction_text="Infosys",
                attributes={"start": 11, "end": 18}
            ),
            le.data.Extraction(
                extraction_class="AMOUNT",
                extraction_text="Rs. 5 lakh",
                attributes={"start": 26, "end": 36}
            ),
            le.data.Extraction(
                extraction_class="QTY",
                extraction_text="200 chairs",
                attributes={"start": 41, "end": 51}
            ),
            le.data.Extraction(
                extraction_class="GPE",
                extraction_text="Bangalore",
                attributes={"start": 55, "end": 64}
            )
        ]
    )
]


In [54]:
class_colors = {
    "PERSON": "red",
    "ORG": "green",
    "DATE": "blue",
    "GPE": "orange",
    "AMOUNT":"grey",
    "QTY":"brown"
}


In [21]:
result = le.extract(
    text_or_documents=text,
    prompt_description=prompt,
    examples=examples,
    model_id="gemini-2.5-flash",
)

In [55]:
html_output = "<h3>Extracted Entities</h3><ul><br>"
for ent in result.extractions:
    ex_class=ent.extraction_class
    color = class_colors[ex_class]
    html_output += f"""
    <li>
        <span style="background-color:{color}; padding:2px 5px; border-radius:4px;">
            <b>{ent.extraction_text}</b>
        </span>
        ({ex_class}) → [{ent.char_interval.start_pos}, {ent.char_interval.end_pos}]
    </li>
    <br>
    """
html_output += "</ul>"

display(HTML(html_output))